# 케라스 보스턴 주택 가격 모델

### modules import

In [2]:
import tensorflow as tf 
from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.utils import plot_model 

from sklearn.model_selection import train_test_split 

import numpy as np
import matplotlib.pyplot as plt 
plt.style.use('seaborn-white')

### 데이터 로드
- 데이터의 수가 상당히 적기 때문에 테스트 데이터의 비율을 20%로 지정

- 13개의 특성을 가짐

- 각각의 특성이 모두 다른 스케일, 즉 단위가 모두 다름
  - 범죄율: 0~1 사이의 값
  - 방의 개수 3~9 사이의 값

- 정답 레이블은 주택 가격의 중간가격($1000 단위)

In [3]:
tf.random.set_seed(111)

(x_train_full, y_train_full), (x_test, y_test) = load_data(path='boston_housing.npz',
                                                           test_split=0.2,
                                                           seed=111)

57026/57026 [==============================] - 0s 0us/step


### 데이터 확인

In [4]:
print('학습 데이터: {}\t레이블: {}'.format(x_train_full.shape, y_train_full.shape))
print('테스트 데이터: {}\t레이블: {}'.format(x_test.shape, y_test.shape))

학습 데이터: (404, 13)	레이블: (404,)
테스트 데이터: (102, 13)	레이블: (102,)


In [5]:
print(x_train_full[0])

[2.8750e-02 2.8000e+01 1.5040e+01 0.0000e+00 4.6400e-01 6.2110e+00
 2.8900e+01 3.6659e+00 4.0000e+00 2.7000e+02 1.8200e+01 3.9633e+02
 6.2100e+00]


In [6]:
print(y_train_full[0])  # $25.0 ---> $25,000

25.0


### 데이터 전처리
- Standardization

- 특성의 단위가 모두 다르기 때문에 **동일한 범위로 조정**

In [7]:
mean = np.mean(x_train_full, axis=0)
std = np.std(x_train_full,axis=0)

x_train_preprocessed = (x_train_full - mean) /std
x_test = (x_test - mean) / std 

x_train, x_val, y_train, y_val = train_test_split(x_train_preprocessed, y_train_full,
                                                  test_size=0.3,
                                                  random_state=111)

In [8]:
print('학습 데이터: {}\t레이블: {}'.format(x_train_full.shape, y_train_full.shape))
print('학습 데이터: {}\t레이블: {}'.format(x_train.shape, y_train.shape))
print('검증 데이터: {}\t레이블: {}'.format(x_val.shape, y_val.shape))
print('테스트 데이터: {}\t레이블: {}'.format(x_test.shape, y_test.shape))

학습 데이터: (404, 13)	레이블: (404,)
학습 데이터: (282, 13)	레이블: (282,)
검증 데이터: (122, 13)	레이블: (122,)
테스트 데이터: (102, 13)	레이블: (102,)


### 모델 구성
- 학습 데이터가 매우 적은 경우에 모델의 깊이를 깊게 할수록  
  과대적합(Overfitting)이 일어날 확률이 높음

In [9]:
model = Sequential([Dense(100, activation='relu', input_shape=(13, ), name='Dense1'),
                    Dense(64, activation='relu', name='Dense2'),
                    Dense(32, activation='relu', name='Dense3'),
                    Dense(1, name='output')])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense1 (Dense)              (None, 100)               1400      
                                                                 
 Dense2 (Dense)              (None, 64)                6464      
                                                                 
 Dense3 (Dense)              (None, 32)                2080      
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 9,977
Trainable params: 9,977
Non-trainable params: 0
_________________________________________________________________


In [11]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### 모델 컴파일(compile)

- 회귀 문제에서는 주로 평균제곱오차(MSE, Mean Squared Error)를 손실함수로,  
  평균절대오차(MAE, Mean Absolute Error)를 평가지표로 많이 사용!

In [12]:
model.compile(loss='mse',
              optimizer=Adam(learning_rate=1e-2),
              metrics=['mae'])

### 모델 학습

In [13]:
history = model.fit(x_train, y_train, epochs=300,
                    validation_data=(x_val, y_val))

Epoch 1/300
9/9 [==============================] - 1s 20ms/step - loss: 415.9695 - mae: 18.2318 - val_loss: 94.9696 - val_mae: 8.0243
Epoch 2/300
9/9 [==============================] - 0s 4ms/step - loss: 84.8042 - mae: 6.8105 - val_loss: 65.1927 - val_mae: 6.6496
Epoch 3/300
9/9 [==============================] - 0s 4ms/step - loss: 46.5302 - mae: 5.2406 - val_loss: 38.7048 - val_mae: 4.7977
Epoch 4/300
9/9 [==============================] - 0s 4ms/step - loss: 28.6627 - mae: 4.0903 - val_loss: 26.2360 - val_mae: 3.7657
Epoch 5/300
9/9 [==============================] - 0s 4ms/step - loss: 22.0629 - mae: 3.4667 - val_loss: 15.7666 - val_mae: 3.1707
Epoch 6/300
9/9 [==============================] - 0s 4ms/step - loss: 17.3924 - mae: 2.9792 - val_loss: 15.3478 - val_mae: 3.1316
Epoch 7/300
9/9 [==============================] - 0s 4ms/step - loss: 16.8162 - mae: 2.9127 - val_loss: 15.9743 - val_mae: 2.9992
Epoch 8/300
9/9 [==============================] - 0s 4ms/step - loss: 15.3924 -

### 모델 평가 
- `evaluate()`

In [14]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 0s 1ms/step - loss: 15.8466 - mae: 2.6733


[15.846640586853027, 2.673264503479004]

In [15]:
print(history.history.keys())

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


In [ ]:
history_dict = history.history

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1,2,1)
ax1.plot(epochs, loss, color='blue', label='train_loss')
ax1.plot(epochs, val_loss, color='red', label='val_loss')
ax1.set_title('Train and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.grid()
ax1.legend()

mae = history_dict['mae']
val_mae = history_dict['val_mae']

ax2 = fig.add_subplot(1,2,2)
ax2.plot(epochs, mae, color='blue', label='train_mae')
ax2.plot(epochs, val_mae, color='red', label='val_mae')
ax2.set_title('Train and Validation MAE')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('MAE')
ax2.grid()
ax2.legend()

### K-Fold 교차 검증

- 데이터셋의 크기가 매우 작은 경우에  
  [훈련, 검증, 테스트] 데이터로 나누게 되면 과소적합이 일어날 확률이 높음

- 이를 해결하기 위해 K-Fold 교차 검증 실행
  <br>

  <img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width="600">

  <sub>출처: https://scikit-learn.org/stable/modules/cross_validation.html</sub>

### 모델 재구성